In [3]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.3.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [105]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [8]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)


# Show the delayed flight data.
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [29]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')




In [31]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
data_df = """
SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms=4
GROUP BY year
ORDER BY year DESC"""

spark.sql(data_df).show()

+----+---------+
|year|avg_price|
+----+---------+
|2022|296363.88|
|2021|301819.44|
|2020|298353.78|
|2019| 300263.7|
+----+---------+



In [32]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

data_built = """
SELECT date_built AS date_data, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms=3
AND bathrooms=3
GROUP BY date_data
ORDER BY date_data DESC"""

spark.sql(data_built).show()

+---------+---------+
|date_data|avg_price|
+---------+---------+
|     2017|292676.79|
|     2016|290555.07|
|     2015| 288770.3|
|     2014|290852.27|
|     2013|295962.27|
|     2012|293683.19|
|     2011|291117.47|
|     2010|292859.62|
+---------+---------+



In [34]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

home_avg = """
SELECT date_built AS date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms=3
AND bathrooms=3
and floors=2
GROUP BY date_built
ORDER BY date_built DESC"""

spark.sql(home_avg).show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2017|282026.59|
|      2016| 287812.6|
|      2015|291788.36|
|      2014|294195.13|
|      2013|295142.13|
|      2012|295858.68|
|      2011|281413.45|
|      2010|280447.23|
+----------+---------+



In [40]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

house_sale = """
SELECT view AS view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE price==350000
GROUP BY view
ORDER BY view DESC"""

spark.sql(house_sale).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
+----+---------+

--- 0.34671497344970703 seconds ---


In [44]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [45]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [127]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

house= ("""
  with allColumns
  (select /*+ BROADCAST(home_sales) */
  a.id,
  a.date,
  a.date_built,
  a.price,
  a.bedrooms,
  a.bathrooms,
  a.sqft_living,
  a.sqft_lot,
  a.floors,
  a.waterfront
  a.view from home_sales
  )""")
spark.sql(house).show()

print("--- %s seconds ---" % (time.time() - start_time))


TypeError: ignored

In [118]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_data")

In [119]:
# 11. Read the parquet formatted data.
p_df = spark.read.parquet('home_sales_data')

In [120]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView("home_sales_data")


In [122]:
home_sales_data.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [125]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql= ("""
SELECT view AS view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales_data
WHERE price = 350000
GROUP BY view
ORDER BY view DESC
""")




print("--- %s seconds ---" % (time.time() - start_time))

--- 6.771087646484375e-05 seconds ---


In [126]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales_data")

TypeError: ignored

In [128]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("home_sales_data")

False